In [27]:
import numpy as np
from Py6S import *
# Create a SixS object called s (used as the standard name by convention)
s = SixS()
s.ground_reflectance = GroundReflectance.HomogeneousLambertian(1)
geom = Geometry.User()
geom.from_time_and_location(lat=52.04, lon=0.76, datetimestring='2020/06/22T12:00',
                            view_z=0, view_a=0)
s.geometry = geom

s.altitudes.set_sensor_satellite_level()
s.wavelength = Wavelength(0.550)
s.run()
s.outputs.write_output_file('test.txt')

In [14]:
wavelengths1, results1 = SixSHelpers.Wavelengths.run_wavelengths(s, np.arange(400,900,10)/1000,
                                                               output_name='pixel_radiance')

Running for many wavelengths - this may take a long time


In [1]:
from pyeosim import datasets
import xarray
import numpy as np
from Py6S import *
# Create a SixS object called s (used as the standard name by convention)
s = SixS()
s.ground_reflectance = GroundReflectance.HomogeneousLambertian(1)
geom = Geometry.User()
geom.from_time_and_location(lat=52.04, lon=0.76, datetimestring='2020/06/22T12:00',
                            view_z=0, view_a=0)
s.geometry = geom

s.altitudes.set_sensor_satellite_level()

# Run the 6S simulation defined by this SixS object across the
# whole VNIR range
wavelengths1, results1 = SixSHelpers.Wavelengths.run_wavelengths(s, np.arange(400,900,2)/1000,
                                                               output_name='pixel_radiance')
# Plot these results, with the y axis label set to "Pixel Radiance"
SixSHelpers.Wavelengths.plot_wavelengths(wavelengths1, results1, "Pixel Radiance")
ref_100 = xarray.DataArray(results1, [('wavelength', wavelengths1*1000)]).interp(
    wavelength = np.arange(400,900,1)
).interpolate_na('wavelength')

rho = np.arange(0,1,.001)
output = np.empty(shape=(len(rho), 500))

for i, _rho in enumerate(rho):
    _res = ref_100* _rho
    output[i,:] = _res

LUT = xarray.DataArray(output,[('rho', rho),('wavelength', np.arange(400,900,1))])
LUTd = LUT.to_dataset(name='pixel_radiance')
LUTd.attrs['6S_output_params'] = str(s.outputs.values)